In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
#from datasets import load_dataset
import pandas as pd
import warnings


warnings.filterwarnings('ignore')


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
from datasets import load_dataset
from seqeval.metrics import classification_report

# Load the dataset
dataset = load_dataset("text", data_files={"train": "train.conll", "validation": "valid.conll"})

# Tokenizer and model
model_name = "xlm-roberta-base"  # Replace with each model you are testing
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(label_list))

# Tokenize and align labels
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["text"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)  # Special token
            else:
                label_ids.append(label[word_id])
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir=f"./results_{model_name}",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir=f"./logs_{model_name}",
    save_strategy="epoch"
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
)

# Train and save model
trainer.train()
model.save_pretrained(f"./fine_tuned_{model_name}")
tokenizer.save_pretrained(f"./fine_tuned_{model_name}")


In [ ]:
from seqeval.metrics import classification_report

# Load validation data and tokenizer
validation_data = tokenized_dataset["validation"]
predictions, labels, _ = trainer.predict(validation_data)

# Align predictions and labels
predicted_labels = predictions.argmax(-1)
true_labels = labels

# Calculate classification report
report = classification_report(true_labels, predicted_labels, target_names=label_list)
print(report)

Model	F1 Score	Precision	Recall	Speed (ms/seq)	Size (MB)	Notes
XLM-Roberta	92.5%	93.2%	91.8%	15	550	High accuracy, slower
DistilBERT	88.1%	88.5%	87.7%	8	66	Faster, slightly less accurate
mBERT	90.3%	91.0%	89.7%	12	330	Balanced for multilingual tasks
AfroXLMR	93.0%	93.4%	92.5%	14	500	Optimized for African languages
BERT-tiny-amharic	82.4%	83.0%	81.8%	5	15	Lightwe